In [1]:
from skimage import io 
import os 
from matplotlib import pyplot as plt
import numpy as np
import matplotlib as mpl

def otsu_thresholding2(img1):
    
    #Kopie des Bildes machen und glätten
    imgT = img1.copy()
    rvl = img1.ravel()
    

    #range und numerische Werte des Histogramms definieren
    ran = round(max(rvl) - min(rvl))
    counts, bins = np.histogram(rvl,bins = ran)
 
    
    
    #leere Liste für alle within class variances erstellen
    variance_list = list()

    # optimalen threshhold herausfinden
    for T in range(1,len(counts)):
        
        #within class variance herausfinden
        
        #definieren von w0 und Hintergrund Mittelwert
        sum_back = 0 
        mean_sum_back = 0
        for i in range(0,T):
            sum_back += counts[i]
            mean_sum_back += counts[i] * bins[i]

        w0 = sum_back / sum(counts)
        mean_back = mean_sum_back / sum_back


        #definieren von w1 und Vordergrund Mittelwert
        sum_obj = 0
        mean_sum_obj = 0

        for j in range(T,len(counts)):
            sum_obj += counts[j]
            mean_sum_obj += counts[j] * bins[j]

        w1 = sum_obj / sum(counts)
        mean_obj = mean_sum_obj / sum_obj

        #Varianz Hintergrund
        var_sum_back = 0
        for k in range(0, T):
            var_sum_back += counts[k] * (bins[k] - mean_back)**2 

        var_back = var_sum_back / sum_back

        #Varianz Vordergrund
        var_sum_obj = 0
        for l in range(T, len(counts)):
            var_sum_obj += counts[l] * (bins[l] - mean_obj)**2 

        var_obj = var_sum_obj / sum_obj

        # Within Class Varianz berechnen
        within_class_varianz = w0 * var_back + w1 * var_obj
        
        #within class variance zu Liste hinzufügen
        variance_list.append(within_class_varianz)
   
    #minimalen Wert für T aussuchen 
    minvar = min(variance_list)
    for m in range(0,len(variance_list)):
        if variance_list[m] == minvar: 
            THRESH = round(bins[m])

    
    #intensitätswerte anpassen
    for p in np.ndindex(imgT.shape):
        if img1[p] < THRESH:
            imgT[p] = 0
        else:
            imgT[p] = 65535
    return imgT


def anwendung(function,input_path,result_path):
    files = os.listdir(input_path)
    for file in files:
        file_path = os.path.join(input_path,file)
        result_file_path = os.path.join(result_path,file)
        result = function(io.imread(file_path))
        io.imsave(result_file_path,result)

In [ ]:
#Otsu 2auf Datensatz 2 ohne preprocessing
anwendung(otsu_thresholding2,'../BilderDaten/BilderDaten/N2DL-HeLa/img/','../Ergebnisse/Otsu/N2DL-HeLa/')

# Otsu 2 auf Datensatz 1 ohne preprocessing
anwendung(otsu_thresholding2,'../BilderDaten/BilderDaten/N2DH-GOWT1/img/','../Ergebnisse/Otsu/N2DH-GOWT1/')

# Otsu auf Datensatz 2 mit Mean Filter

anwendung(otsu_thresholding2,'../Ergebnisse/Mean/N2DL-HeLa/','../Ergebnisse/Mean+Otsu/N2DL-HeLa/')

# Otsu auf Datensatz 1 mit Mean Filter

anwendung(otsu_thresholding2,'../Ergebnisse/Mean/N2DH-GOWT1/','../Ergebnisse/Mean+Otsu/N2DH-GOWT1')

# Otsu auf Datensatz 2 mit Median Filter

anwendung(otsu_thresholding2,'../Ergebnisse/Median/N2DL-HeLa/','../Ergebnisse/Median+Otsu/N2DL-HeLa/')

# Otsu auf Datensatz 1 mit Median Filter

anwendung(otsu_thresholding2,'../Ergebnisse/Median/N2DH-GOWT1/','../Ergebnisse/Median+Otsu/N2DH-GOWT1/')

In [ ]:
# Histogrammstreching für Datensatz 2
import numpy as np
def histogrammstretching2(img):
    a = 0
    b = 65535
    c = min(img.ravel())
    d = max(img.ravel())
    img_copy = img.copy()
    for p in np.ndindex(img_copy.shape):
        img_copy[p] = (img[p]-c) * ((b-a)/(d-c))+a
    return img_copy

In [ ]:
# Histogrammstreching für Datensatz 1
import numpy as np
def histogrammstretching1(img):
    a = 0
    b = 255
    c = min(img.ravel())
    d = max(img.ravel())
    img_copy = img.copy()
    for p in np.ndindex(img_copy.shape):
        img_copy[p] = (img[p]-c) * ((b-a)/(d-c))+a
    return img_copy

In [ ]:
# Anwendung Histogrammstreching auf Datensatz 1

anwendung(histogrammstretching1,'../BilderDaten/BilderDaten/N2DH-GOWT1/img/','../Ergebnisse/Histogramm_stretching/N2DH-GOWT1/')

# Anwendung Histogrammstreching auf Datensatz 2

anwendung(histogrammstretching2,'../BilderDaten/BilderDaten/N2DL-HeLa/img/','../Ergebnisse/Histogramm_stretching/N2DL-HeLa/')

# Otsu auf Histogrammstreching Datensatz 2

anwendung(otsu_thresholding2,'../Ergebnisse/Histogramm_stretching/N2DL-HeLa/','../Ergebnisse/Histogramm+Otsu/H2DL-HeLa/')


In [ ]:
#Probleme mit den gt Bildern bei Datensatz 1 und 2 (weil tif Dateien)
# Versuch diese zu ändern -> Werte auf 0 und 255 setzen 

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 65535
        else:
            img[o] = 0
    
    return img

#Anwenden auf Datensatz 1 beispiel 

gt1 = binarize(io.imread('../BilderDaten/BilderDaten/N2DH-GOWT1/gt/man_seg01.tif'))

plt.hist(gt1)
plt.show()


In [ ]:
# gt Bilder Datensatz 1 auf 0 und 255 setzen 

anwendung(binarize,'../BilderDaten/BilderDaten/N2DH-GOWT1/gt/','../BilderDaten/BilderDaten/N2DH-GOWT1/gt_binarized/')

# gt Bilder Datensatz 2 auf 0 und 255 setzten 

anwendung(binarize, '../BilderDaten/BilderDaten/N2DL-HeLa/gt/','../BilderDaten/BilderDaten/N2DL-HeLa/gt_binarized/')
